# Data Exploration and NLP Modeling 
## By BROSSEAU Alexandre & COGORDAN Alexandre

In [48]:
import nltk
import requests
import time
import pandas as pd
import os
import re
import gensim
import gensim.corpora as corpora
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import streamlit as st
import numpy as np
import tensorflow as tf
import tensorboard as tb

from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cosine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from collections import Counter
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
from dotenv import find_dotenv, load_dotenv
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, Word2Vec
from sklearn.manifold import TSNE
from tensorboard.plugins import projector

load_dotenv()

True

## Web scraping

### We get the requests and the dataframe we've created so far

In [49]:
df = pd.read_csv('reviews.csv', usecols=['text', 'rating', 'location'])

### We call our API key to start web scraping

In [50]:
api_key = os.getenv('YELP_API_KEY')
headers = {'Authorization': 'Bearer ' + api_key}
# headers = {'Authorization': 'Bearer ' + 'rjj-CYij9D5rBs0P7jLtSGO6u2hBp38A5jMZTGELqjvvojBqfepQac4jprE6LhD84IuHE9UuswLpOL46Ou8IA04Jd4Ije97e5rv5Q4Ifag8tQrMlJirC2-eTUH58ZXYx'}

### We get the businesses' IDs

In [51]:
def get_all_business_data(base_url):
    
    #all_business_ids = []
    list_of_businesses = []

    while True:
        response = requests.get(base_url, headers=headers)
        if response.status_code != 200:
            break  

        data = response.json()
        businesses = data.get('businesses', [])

        # business_info = data.get('businesses')

        if not businesses:
            break  # Break the loop if no more businesses are returned

        for business in businesses:
            #business_id = business.get('id') # 'name', 'price', 'url', 'review_count', 'display_address', 'image_url', 'display_phone', 'categories'
            business_dict = {'restaurant_id': business.get('id'), 'business_name': business.get('name'), 'business_price': business.get('price'), 'business_url': business.get('url'), 'business_review_count': business.get('review_count'), 'business_display_address': business.get('location').get('display_address'), 'business_image_url': business.get('image_url'), 'business_display_phone': business.get('display_phone'), 'business_categories': business.get('categories')}
            list_of_businesses.append(business_dict)
            #if business_id:
                #all_business_ids.append(business_id)

        # Update the offset in the URL for the next request
        if 'offset=' in base_url:
            base_url = base_url.rsplit('offset=', 1)[0] + f'offset={len(list_of_businesses)}'
        else:
            base_url += f'&offset={len(list_of_businesses)}'

        time.sleep(1)  

    return list_of_businesses

### We get the reviews from the business

In [52]:
def get_reviews(business_data, city):
    restaurant_ids = [restaurant['restaurant_id'] for restaurant in business_data]

    business_df = pd.DataFrame()
    count = 0
    
    for restaurant_id in restaurant_ids:
        url2 = "https://api.yelp.com/v3/businesses/" + restaurant_id + "/reviews?sort_by=yelp_sort"
        response = requests.get(url2, headers=headers)
        reviews_data = response.json()
        
        try:
            for review in reviews_data['reviews']:
                business_df.loc[count, 'text'] = review['text']
                business_df.loc[count, 'rating'] = review['rating']
                business_df.loc[count, 'location'] = city
                business_df.loc[count, 'restaurant_id'] = restaurant_id
                count += 1

        except:
            print("No reviews for this restaurant")
        
    return business_df

In [53]:
# cities = ['San+Diego', 'San+Jose', 'Seattle', 'Maryville', 'Salt+Lake+City', 'Oklahoma+City', 'Austin', 'Louisville', 'Indianapolis', 'Portland', 'Santa+Cruz', 'Jacksonville', 'Hudson', 'Dallas', 'Phoenix', 'Asheville']
# 'Memphis', 'Boston', 'Seattle', 'Denver', 'Washington', 'Nashville', 'Baltimore', 'Oklahoma+City', 'Louisville', 'Portland', 'Las+Vegas', 'Milwaukee', 'Albuquerque', 'Tucson', 'Fresno', 'Sacramento', 'Long+Beach', 'Kansas+City', 'Mesa'
# 'Atlanta', 'Raleigh', 'Miami', 'Omaha', 'Oakland', 'Tulsa', 'Minneapolis', 'Cleveland', 'Wichita', 'New+York', 'Los+Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', 'San+Antonio', 'San+Diego', 'Dallas', 'San+Jose', 'Austin', 'Jacksonville', 'Indianapolis', 'San+Francisco', 'Columbus', 'Fort+Worth', 'Charlotte', 'Detroit', 'El+Paso', 'Arlington', 'New+Orleans', 'Bakersfield', 'Tampa', 'Honolulu', 'Aurora', 'Anaheim', 'Santa+Ana', 'St.+Louis',
# 'Riverside', 'Corpus+Christi', 'Lexington', 'Pittsburgh', 'Anchorage', 'Stockton', 'Cincinnati', 'Saint+Paul', 'Toledo', 'Newark', 'Greensboro', 'Plano', 'Henderson', 'Lincoln', 'Buffalo', 'Fort+Wayne', 'Jersey+City', 'Chula+Vista', 'Orlando', 'St.+Petersburg', 'Norfolk', 'Chandler', 'Laredo', 'Madison', 'Durham', 'Lubbock', 'Winston-Salem', 'Garland', 'Glendale', 'Hialeah', 'Reno',
# 'Baton+Rouge', 'Irvine', 'Chesapeake', 'Irving', 'Scottsdale', 'North+Las+Vegas', 'Fremont', 'Gilbert', 'San+Bernardino', 'Boise', 'Birmingham', 'Rochester', 'Richmond', 'Spokane', 'Des+Moines', 'Montgomery', 'Modesto', 'Fayetteville', 'Tacoma', 'Shreveport', 'Fontana', 'Oxnard', 'Aurora', 'Moreno+Valley', 'Akron', 'Yonkers', 'Columbus', 'Augusta', 'Little+Rock', 'Amarillo', 'Mobile', 'Huntington+Beach', 'Glendale', 'Grand+Rapids', 'Salt+Lake+City', 'Tallahassee', 'Huntsville', 'Worcester', 'Knoxville', 'Grand+Prairie', 'Newport+News', 'Brownsville', 'Santa+Clarita', 'Overland+Park', 'Providence', 'Garden+Grove', 'Chattanooga', 'Oceanside', 'Santa+Rosa', 'Fort+Lauderdale',
# 'Rancho+Cucamonga', 'Port+St.+Lucie', 'Ontario', 'Vancouver', 'Tempe', 'Springfield', 'Lancaster', 'Eugene', 'Pembroke+Pines', 'Salem', 'Cape+Coral', 'Peoria', 'Sioux+Falls', 'Springfield', 'Elk+Grove', 'Rockford', 'Palmdale', 'Corona', 'Salinas', 'Pomona', 'Pasadena', 'Joliet', 'Paterson', 'Kansas+City', 'Torrance', 
# 'Syracuse', 'Bridgeport', 'Hayward', 'Fort+Collins', 'Escondido', 'Lakewood', 'Naperville', 'Dayton', 'Hollywood', 'Sunnyvale', 'Alexandria', 'Mesquite', 'Hampton', 'Pasadena', 'Orange', 'Savannah', 'Cary', 'Fullerton', 'Warren', 'Clarksville', 'McKinney', 'McAllen', 'New+Haven', 'Sterling+Heights', 'West+Valley+City', 'Columbia', 'Killeen', 'Topeka', 'Thousand+Oaks', 'Cedar+Rapids', 'Olathe', 'Elizabeth', 'Waco', 'Hartford', 'Visalia', 'Gainesville', 'Simi+Valley',

cities = [
    'Stamford', 'Bellevue', 'Concord', 'Miramar', 'Coral+Springs',
    'Lafayette', 'Charleston', 'Carrollton', 'Roseville', 'Thornton', 'Beaumont', 'Allentown', 'Surprise', 'Evansville',
    'Abilene', 'Frisco', 'Independence', 'Santa+Clara', 'Springfield', 'Vallejo', 'Victorville', 'Athens', 'Peoria',
    'Lansing', 'Ann+Arbor', 'El+Monte', 'Denton', 'Berkeley', 'Provo', 'Downey', 'Midland', 'Norman', 'Waterbury',
    'Costa+Mesa', 'Inglewood', 'Manchester', 'Murfreesboro', 'Columbia', 'Elgin', 'Clearwater', 'Miami+Gardens',
    'Rochester', 'Pueblo', 'Lowell', 'Wilmington', 'Arvada', 'Ventura', 'Westminster', 'West+Covina', 'Gresham',
    'Fargo', 'Norwalk', 'Carlsbad', 'Fairfield', 'Cambridge', 'Wichita+Falls', 'High+Point', 'Billings', 'Green+Bay',
    'West+Jordan', 'Richmond', 'Murrieta', 'Burbank', 'Palm+Bay', 'Everett', 'Flint', 'Antioch', 'Erie', 'South+Bend',
    'Daly+City', 'Centennial', 'Temecula', 'Rialto', 'Thornton', 'El+Paso', 'San+Mateo', 'Midland', 'Davenport', 'Santa+Monica',
    'Sandy+Springs', 'Boulder', 'Hillsboro', 'Frisco', 'Greeley', 'San+Bernardino', 'Jurupa+Valley', 'Kenosha', 'Rochester',
    'Olathe', 'Newport+Beach', 'Topeka', 'Athens', 'Santa+Clarita', 'Simi+Valley', 'Columbia', 'Concord', 'Lafayette',
    'Charleston', 'Carrollton', 'Roseville', 'Thornton'
]

business_df = pd.DataFrame()
business_data = pd.DataFrame()

for city in cities:
    url = (f'https://api.yelp.com/v3/businesses/search?location={city}&term=restaurants&categories=french&price=3&price=4&sort_by=best_match')
    business_id = get_all_business_data(url)
    business_data = pd.concat([business_data, pd.DataFrame(business_id)], ignore_index=True)
    business_df = pd.concat([business_df, get_reviews(business_id, city)], ignore_index=True)

/var/folders/p9/flhwy6kx3s75yh91rftjm9dr0000gn/T/ipykernel_52573/2927359860.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'The dinner we had here was absolutely perfect in every way! Everything from the phone call confirming our reservation.... to our warm welcome upon...' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  business_df.loc[count, 'text'] = review['text']
/var/folders/p9/flhwy6kx3s75yh91rftjm9dr0000gn/T/ipykernel_52573/2927359860.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Syracuse' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  business_df.loc[count, 'location'] = city
/var/folders/p9/flhwy6kx3s75yh91rftjm9dr0000gn/T/ipykernel_52573/2927359860.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise 

No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No reviews for this restaurant
No revie

Il faudra juste faire un merge demain!

In [54]:
business_df['location'].unique()

array(['Syracuse', 'Bridgeport', 'Fort+Collins', 'Escondido', 'Lakewood',
       'Naperville', 'Dayton', 'Hollywood', 'Sunnyvale', 'Alexandria',
       'Mesquite', 'Hampton', 'Pasadena', 'Orange', 'Savannah', 'Cary',
       'Fullerton', 'Warren', 'Clarksville', 'McKinney', 'New+Haven',
       'Sterling+Heights', 'West+Valley+City', 'Columbia',
       'Thousand+Oaks', 'Cedar+Rapids', 'Olathe', 'Hartford', 'Visalia',
       'Gainesville', 'Simi+Valley'], dtype=object)

In [55]:
business_df.to_csv('df-export-seven.csv', index=False)
business_data.to_csv('id-export-seven.csv', index=False)

### Merge

In [153]:
ouput_dfs = []

cities = ['new_orleans', 'nyc', 'chicago', 'los_angeles', 'sf', 'philadelphia', 'las_vegas', 'houston', 'phoenix', 'miami']

for city in cities:
    reviews_list = globals()[f'{city}_list_of_reviews']
    ouput_df = pd.DataFrame(reviews_list, columns=['text', 'rating', 'location'])
    ouput_dfs.append(ouput_df)

output = pd.concat(ouput_dfs, ignore_index=True)
df = pd.concat([df, output], ignore_index=True)

In [157]:
df.drop_duplicates(inplace=True)
df['rating'].value_counts()

rating
5    367
4    134
3     69
2     29
1     19
Name: count, dtype: int64

In [56]:
df_data = pd.read_csv('df-export-seven.csv')
df_id = pd.read_csv('id-export-seven.csv')

In [57]:
df_data

,text,rating,location,restaurant_id
0,The dinner we had here was absolutely perfect ...,5.0,Syracuse,C9zPmCBRyUHYRAk2LcrGEg
1,You know you have found someplace special when...,5.0,Syracuse,C9zPmCBRyUHYRAk2LcrGEg
2,My husband surprised me with a date night and ...,5.0,Syracuse,C9zPmCBRyUHYRAk2LcrGEg
3,Amazing place and food and wine selection!!! G...,5.0,Syracuse,HtRzC4htDXvLf2Mcl6k52g
4,3.5 stars and a belated review. Came here in A...,3.0,Syracuse,HtRzC4htDXvLf2Mcl6k52g
...,...,...,...,...
1249,Terrible! This restaurant has really gone down...,1.0,Simi+Valley,v_49OmxMP9JOGDuca2atPA
1250,This is a very swanky French restaurant in San...,4.0,Simi+Valley,v_49OmxMP9JOGDuca2atPA
1251,"Wow, where to begin with how excellent our exp...",5.0,Simi+Valley,1M4U8BZzWRcV1ZRw1SUjwA
1252,The Dock is simple elegant restaurant with a b...,5.0,Simi+Valley,1M4U8BZzWRcV1ZRw1SUjwA


In [60]:
df_id

,restaurant_id,business_name,business_price,business_url,business_review_count,business_display_address,business_image_url,business_display_phone,business_categories
0,C9zPmCBRyUHYRAk2LcrGEg,Joelle's French Bistro,$$$,https://www.yelp.com/biz/joelles-french-bistro...,44,"['4423 State Street Rd', 'Skaneateles, NY 13152']",https://s3-media2.fl.yelpcdn.com/bphoto/B2Awi7...,(315) 685-3063,"[{'alias': 'french', 'title': 'French'}]"
1,HtRzC4htDXvLf2Mcl6k52g,Saint Urban,$$$$,https://www.yelp.com/biz/saint-urban-syracuse?...,82,"['124 Dell St', 'Syracuse, NY 13210']",https://s3-media1.fl.yelpcdn.com/bphoto/I2M2TQ...,(315) 243-8509,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},..."
2,O7KQzEsuixtA6g-IKZkbSQ,Arad Evans Inn,$$$,https://www.yelp.com/biz/arad-evans-inn-fayett...,92,"['7206 E Genesee St', 'Fayetteville, NY 13066']",https://s3-media4.fl.yelpcdn.com/bphoto/KNBrhe...,(315) 637-2020,"[{'alias': 'newamerican', 'title': 'New Americ..."
3,EoeEMRMY9CFd7HY4BxfbQw,Le Fat Poodle,$$$,https://www.yelp.com/biz/le-fat-poodle-old-gre...,146,"['20 Arcadia Rd', 'Old Greenwich, CT 06870']",https://s3-media3.fl.yelpcdn.com/bphoto/p1fYPW...,(203) 717-1515,"[{'alias': 'french', 'title': 'French'}, {'ali..."
4,c1sYo03Fht3Aaz8QrjPUOQ,La Plage Wading River,$$$,https://www.yelp.com/biz/la-plage-wading-river...,230,"['131 Creek Rd', 'Wading River, NY 11792']",https://s3-media4.fl.yelpcdn.com/bphoto/jbj3Rh...,(631) 744-9200,"[{'alias': 'french', 'title': 'French'}, {'ali..."
...,...,...,...,...,...,...,...,...,...
408,a3Y0KJZ0Xl3gzRysPO4prg,Wheatleigh,$$$$,https://www.yelp.com/biz/wheatleigh-lenox?adju...,44,"['Hawthorne Rd', 'Lenox, MA 01240']",https://s3-media4.fl.yelpcdn.com/bphoto/zcYMTv...,(844) 704-8337,"[{'alias': 'hotels', 'title': 'Hotels'}, {'ali..."
409,t-TNzDrBcPplpMhL59OZ7w,Cafe Lafayette,$$$,https://www.yelp.com/biz/cafe-lafayette-exeter...,156,"['151 South E St', 'Ste A', 'Exeter, CA 93221']",https://s3-media3.fl.yelpcdn.com/bphoto/ShInwH...,(559) 592-9463,"[{'alias': 'french', 'title': 'French'}, {'ali..."
410,nrcZy-7pBEgfL4NxOaA8Aw,La Cuisine French Restaurant,$$$,https://www.yelp.com/biz/la-cuisine-french-res...,157,"['48 SW 1st Ave', 'Ocala, FL 34471']",https://s3-media3.fl.yelpcdn.com/bphoto/ox_Rvz...,(352) 433-2570,"[{'alias': 'french', 'title': 'French'}]"
411,nyk7ga8gqmnO8e3ILyrSvg,Orsay,$$$,https://www.yelp.com/biz/orsay-jacksonville-4?...,1105,"['3630 Park St', 'Jacksonville, FL 32205']",https://s3-media1.fl.yelpcdn.com/bphoto/xypA2Y...,(904) 381-0909,"[{'alias': 'french', 'title': 'French'}, {'ali..."


In [59]:
df_id.drop_duplicates(subset=['restaurant_id'], inplace=True)

In [61]:
merged_df = df_data.merge(df_id, on='restaurant_id', how='left')

In [62]:
merged_df

,text,rating,location,restaurant_id,business_name,business_price,business_url,business_review_count,business_display_address,business_image_url,business_display_phone,business_categories
0,The dinner we had here was absolutely perfect ...,5.0,Syracuse,C9zPmCBRyUHYRAk2LcrGEg,Joelle's French Bistro,$$$,https://www.yelp.com/biz/joelles-french-bistro...,44,"['4423 State Street Rd', 'Skaneateles, NY 13152']",https://s3-media2.fl.yelpcdn.com/bphoto/B2Awi7...,(315) 685-3063,"[{'alias': 'french', 'title': 'French'}]"
1,You know you have found someplace special when...,5.0,Syracuse,C9zPmCBRyUHYRAk2LcrGEg,Joelle's French Bistro,$$$,https://www.yelp.com/biz/joelles-french-bistro...,44,"['4423 State Street Rd', 'Skaneateles, NY 13152']",https://s3-media2.fl.yelpcdn.com/bphoto/B2Awi7...,(315) 685-3063,"[{'alias': 'french', 'title': 'French'}]"
2,My husband surprised me with a date night and ...,5.0,Syracuse,C9zPmCBRyUHYRAk2LcrGEg,Joelle's French Bistro,$$$,https://www.yelp.com/biz/joelles-french-bistro...,44,"['4423 State Street Rd', 'Skaneateles, NY 13152']",https://s3-media2.fl.yelpcdn.com/bphoto/B2Awi7...,(315) 685-3063,"[{'alias': 'french', 'title': 'French'}]"
3,Amazing place and food and wine selection!!! G...,5.0,Syracuse,HtRzC4htDXvLf2Mcl6k52g,Saint Urban,$$$$,https://www.yelp.com/biz/saint-urban-syracuse?...,82,"['124 Dell St', 'Syracuse, NY 13210']",https://s3-media1.fl.yelpcdn.com/bphoto/I2M2TQ...,(315) 243-8509,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},..."
4,3.5 stars and a belated review. Came here in A...,3.0,Syracuse,HtRzC4htDXvLf2Mcl6k52g,Saint Urban,$$$$,https://www.yelp.com/biz/saint-urban-syracuse?...,82,"['124 Dell St', 'Syracuse, NY 13210']",https://s3-media1.fl.yelpcdn.com/bphoto/I2M2TQ...,(315) 243-8509,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1249,Terrible! This restaurant has really gone down...,1.0,Simi+Valley,v_49OmxMP9JOGDuca2atPA,Pasjoli,$$$$,https://www.yelp.com/biz/pasjoli-santa-monica?...,379,"['2732 Main St', 'Santa Monica, CA 90405']",https://s3-media1.fl.yelpcdn.com/bphoto/f13jW2...,(424) 330-0020,"[{'alias': 'french', 'title': 'French'}]"
1250,This is a very swanky French restaurant in San...,4.0,Simi+Valley,v_49OmxMP9JOGDuca2atPA,Pasjoli,$$$$,https://www.yelp.com/biz/pasjoli-santa-monica?...,379,"['2732 Main St', 'Santa Monica, CA 90405']",https://s3-media1.fl.yelpcdn.com/bphoto/f13jW2...,(424) 330-0020,"[{'alias': 'french', 'title': 'French'}]"
1251,"Wow, where to begin with how excellent our exp...",5.0,Simi+Valley,1M4U8BZzWRcV1ZRw1SUjwA,The Dock,$$$,https://www.yelp.com/biz/the-dock-newport-beac...,563,"['2816 Lafayette Ave', 'Newport Beach, CA 92663']",https://s3-media2.fl.yelpcdn.com/bphoto/MrS8Bb...,(949) 673-3625,"[{'alias': 'newamerican', 'title': 'New Americ..."
1252,The Dock is simple elegant restaurant with a b...,5.0,Simi+Valley,1M4U8BZzWRcV1ZRw1SUjwA,The Dock,$$$,https://www.yelp.com/biz/the-dock-newport-beac...,563,"['2816 Lafayette Ave', 'Newport Beach, CA 92663']",https://s3-media2.fl.yelpcdn.com/bphoto/MrS8Bb...,(949) 673-3625,"[{'alias': 'newamerican', 'title': 'New Americ..."


## Merging with pre-recorded data

In [63]:
output_df = pd.read_csv('reviews.csv')

In [64]:
output_df

,text,rating,location,restaurant_id,business_name,business_price,business_url,business_review_count,business_display_address,business_image_url,business_display_phone,business_categories
0,"Amazing food, service, and ambiance! \n\nMy fr...",5.0,San+Diego,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,$$$,https://www.yelp.com/biz/the-smoking-goat-san-...,1758,"['3408 30th St', 'San Diego, CA 92104']",https://s3-media3.fl.yelpcdn.com/bphoto/GNINLV...,(619) 955-5295,"[{'alias': 'french', 'title': 'French'}, {'ali..."
1,Casual enough place to pop in and get a table ...,4.0,San+Diego,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,$$$,https://www.yelp.com/biz/the-smoking-goat-san-...,1758,"['3408 30th St', 'San Diego, CA 92104']",https://s3-media3.fl.yelpcdn.com/bphoto/GNINLV...,(619) 955-5295,"[{'alias': 'french', 'title': 'French'}, {'ali..."
2,3.5 stars...\n\nArrived for our reservation an...,3.0,San+Diego,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,$$$,https://www.yelp.com/biz/the-smoking-goat-san-...,1758,"['3408 30th St', 'San Diego, CA 92104']",https://s3-media3.fl.yelpcdn.com/bphoto/GNINLV...,(619) 955-5295,"[{'alias': 'french', 'title': 'French'}, {'ali..."
3,"Great brunch experience here at Parc Bistro, b...",5.0,San+Diego,_zFYC91bOjXC9EdsT8VnVQ,Parc Bistro-Brasserie,$$$,https://www.yelp.com/biz/parc-bistro-brasserie...,777,"['2760 5th Ave', 'San Diego, CA 92103']",https://s3-media2.fl.yelpcdn.com/bphoto/nXMP3P...,(619) 795-1501,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a..."
4,Brunch today was borderline disastrous. Thing...,3.0,San+Diego,_zFYC91bOjXC9EdsT8VnVQ,Parc Bistro-Brasserie,$$$,https://www.yelp.com/biz/parc-bistro-brasserie...,777,"['2760 5th Ave', 'San Diego, CA 92103']",https://s3-media2.fl.yelpcdn.com/bphoto/nXMP3P...,(619) 795-1501,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a..."
...,...,...,...,...,...,...,...,...,...,...,...,...
5756,"This charming French restaurant, Pescadou, off...",5.0,Torrance,LrGmxyCqlgg25qZweMZOvg,Pescadou Bistro,$$$,https://www.yelp.com/biz/pescadou-bistro-newpo...,327,"['3325 Newport Blvd', 'Newport Beach, CA 92663']",https://s3-media3.fl.yelpcdn.com/bphoto/szEsa2...,(949) 675-6990,"[{'alias': 'french', 'title': 'French'}]"
5757,"As usual for a French bistro, the service was ...",5.0,Torrance,LrGmxyCqlgg25qZweMZOvg,Pescadou Bistro,$$$,https://www.yelp.com/biz/pescadou-bistro-newpo...,327,"['3325 Newport Blvd', 'Newport Beach, CA 92663']",https://s3-media3.fl.yelpcdn.com/bphoto/szEsa2...,(949) 675-6990,"[{'alias': 'french', 'title': 'French'}]"
5758,Snagged an 8:00pm reservation here on a Friday...,5.0,Torrance,iTLSVLEcErPNQW2k_iGr_g,Sur Restaurant & Lounge,$$$,https://www.yelp.com/biz/sur-restaurant-and-lo...,4172,"['606 N Robertson Blvd', 'West Hollywood, CA 9...",https://s3-media2.fl.yelpcdn.com/bphoto/bHrMyq...,(310) 289-2824,"[{'alias': 'newamerican', 'title': 'New Americ..."
5759,Me and my husband ate dinner here this past tu...,5.0,Torrance,iTLSVLEcErPNQW2k_iGr_g,Sur Restaurant & Lounge,$$$,https://www.yelp.com/biz/sur-restaurant-and-lo...,4172,"['606 N Robertson Blvd', 'West Hollywood, CA 9...",https://s3-media2.fl.yelpcdn.com/bphoto/bHrMyq...,(310) 289-2824,"[{'alias': 'newamerican', 'title': 'New Americ..."


In [65]:
output_df = pd.concat([output_df, merged_df], ignore_index=True)

In [66]:
output_df

,text,rating,location,restaurant_id,business_name,business_price,business_url,business_review_count,business_display_address,business_image_url,business_display_phone,business_categories
0,"Amazing food, service, and ambiance! \n\nMy fr...",5.0,San+Diego,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,$$$,https://www.yelp.com/biz/the-smoking-goat-san-...,1758,"['3408 30th St', 'San Diego, CA 92104']",https://s3-media3.fl.yelpcdn.com/bphoto/GNINLV...,(619) 955-5295,"[{'alias': 'french', 'title': 'French'}, {'ali..."
1,Casual enough place to pop in and get a table ...,4.0,San+Diego,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,$$$,https://www.yelp.com/biz/the-smoking-goat-san-...,1758,"['3408 30th St', 'San Diego, CA 92104']",https://s3-media3.fl.yelpcdn.com/bphoto/GNINLV...,(619) 955-5295,"[{'alias': 'french', 'title': 'French'}, {'ali..."
2,3.5 stars...\n\nArrived for our reservation an...,3.0,San+Diego,YLV7ZZoyOAXW3eqidGXBZA,The Smoking Goat,$$$,https://www.yelp.com/biz/the-smoking-goat-san-...,1758,"['3408 30th St', 'San Diego, CA 92104']",https://s3-media3.fl.yelpcdn.com/bphoto/GNINLV...,(619) 955-5295,"[{'alias': 'french', 'title': 'French'}, {'ali..."
3,"Great brunch experience here at Parc Bistro, b...",5.0,San+Diego,_zFYC91bOjXC9EdsT8VnVQ,Parc Bistro-Brasserie,$$$,https://www.yelp.com/biz/parc-bistro-brasserie...,777,"['2760 5th Ave', 'San Diego, CA 92103']",https://s3-media2.fl.yelpcdn.com/bphoto/nXMP3P...,(619) 795-1501,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a..."
4,Brunch today was borderline disastrous. Thing...,3.0,San+Diego,_zFYC91bOjXC9EdsT8VnVQ,Parc Bistro-Brasserie,$$$,https://www.yelp.com/biz/parc-bistro-brasserie...,777,"['2760 5th Ave', 'San Diego, CA 92103']",https://s3-media2.fl.yelpcdn.com/bphoto/nXMP3P...,(619) 795-1501,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7010,Terrible! This restaurant has really gone down...,1.0,Simi+Valley,v_49OmxMP9JOGDuca2atPA,Pasjoli,$$$$,https://www.yelp.com/biz/pasjoli-santa-monica?...,379,"['2732 Main St', 'Santa Monica, CA 90405']",https://s3-media1.fl.yelpcdn.com/bphoto/f13jW2...,(424) 330-0020,"[{'alias': 'french', 'title': 'French'}]"
7011,This is a very swanky French restaurant in San...,4.0,Simi+Valley,v_49OmxMP9JOGDuca2atPA,Pasjoli,$$$$,https://www.yelp.com/biz/pasjoli-santa-monica?...,379,"['2732 Main St', 'Santa Monica, CA 90405']",https://s3-media1.fl.yelpcdn.com/bphoto/f13jW2...,(424) 330-0020,"[{'alias': 'french', 'title': 'French'}]"
7012,"Wow, where to begin with how excellent our exp...",5.0,Simi+Valley,1M4U8BZzWRcV1ZRw1SUjwA,The Dock,$$$,https://www.yelp.com/biz/the-dock-newport-beac...,563,"['2816 Lafayette Ave', 'Newport Beach, CA 92663']",https://s3-media2.fl.yelpcdn.com/bphoto/MrS8Bb...,(949) 673-3625,"[{'alias': 'newamerican', 'title': 'New Americ..."
7013,The Dock is simple elegant restaurant with a b...,5.0,Simi+Valley,1M4U8BZzWRcV1ZRw1SUjwA,The Dock,$$$,https://www.yelp.com/biz/the-dock-newport-beac...,563,"['2816 Lafayette Ave', 'Newport Beach, CA 92663']",https://s3-media2.fl.yelpcdn.com/bphoto/MrS8Bb...,(949) 673-3625,"[{'alias': 'newamerican', 'title': 'New Americ..."


In [67]:
output_df.to_csv('reviews.csv', index=False)